In [144]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)

# Define leg parameters
coxa_len = 28.75
tibia_len = 68.825
femur_len = 40
z_offset = tibia_len

# Define body parameters (for rotational IK)
X0_LEN = 45.768
Y0_LEN = 26.424
Y1_LEN = 52.848
R1_ORIGIN = np.array( (X0_LEN, Y0_LEN, z_offset) )
R2_ORIGIN = np.array( (0.0, Y1_LEN, z_offset) )
R3_ORIGIN = np.array( (-X0_LEN, Y0_LEN, z_offset) )
L1_ORIGIN = np.array( (X0_LEN, -Y0_LEN, z_offset) )
L2_ORIGIN = np.array( (0.0, -Y1_LEN, z_offset) )
L3_ORIGIN = np.array( (-X0_LEN, -Y0_LEN, z_offset) )
leg_origins = np.array( (R1_ORIGIN, R2_ORIGIN, R3_ORIGIN, L3_ORIGIN, L2_ORIGIN, L1_ORIGIN) )
print(leg_origins.shape)

def to_rads(num):
    return num*np.pi/180

def to_degs(num):
    return num/np.pi*180

def pythagoras(nums):
    sums = np.sum(np.power(nums, 2))

    return np.sqrt(sums)

(6, 3)


In [81]:
# Forward kinematics check - make this into a function
def plot_legs(angle):
    leg_base = np.array([0, 0, z_offset], dtype='float32') # xyz
    coxa_end = np.array([0, 0, z_offset], dtype='float32')
    femur_end = np.array([0, 0, 0], dtype='float32')
    tibia_end = np.array([0, 0, 0], dtype='float32')

    # Input desired angles here
    print("\nDesired angles:", ["%0.4f" %i for i in angle])
    angles = angle
    coxa_angle = angles[0] - 90
    femur_angle = angles[1] - 90
    tibia_angle = angles[2]

    theta = tibia_angle + femur_angle - 90

    femur_vert = femur_len * np.cos( to_rads(femur_angle) )
    tibia_vert = tibia_len * np.sin( to_rads(theta) )

    femur_hor = femur_len * np.cos( to_rads(coxa_angle) )
    tibia_hor = tibia_len * np.cos( to_rads(coxa_angle) )

    # # Debug parameters
    # print(z_offset)
    # # Debug desired angles
    # print("%0.4f, %0.4f, %0.4f, %0.4f" %(coxa_angle, femur_angle, tibia_angle, theta))
    # # debug tibia params
    # print("%0.4f, %0.4f" %(femur_vert, tibia_vert))

    # calc coxa end-points
    coxa_end[0] = coxa_len * np.sin( to_rads(coxa_angle) )
    coxa_end[1] = coxa_len * np.cos( to_rads(coxa_angle) )

    # calc femur end-points
    femur_end[0] = coxa_end[0] + femur_vert * np.sin( to_rads(coxa_angle) )
    femur_end[1] = coxa_end[1] + femur_vert * np.cos( to_rads(coxa_angle) )
    femur_end[2] = coxa_end[2] + femur_hor * np.sin( to_rads(femur_angle) )

    # calc tibia end-points
    tibia_end[0] = femur_end[0] + tibia_vert * np.sin( to_rads(coxa_angle) )
    tibia_end[1] = femur_end[1] + tibia_vert * np.cos( to_rads(coxa_angle) )
    tibia_end[2] = femur_end[2] + tibia_hor * np.cos( to_rads( theta-180 ) )

    # sanity check: if all legs are the same length
    print("Coxa length:", pythagoras(coxa_end-leg_base))
    print("Femur length:", pythagoras(femur_end-coxa_end))
    print("Tibia length:", pythagoras(tibia_end-femur_end))

    leg_base_annotate = ["%0.4f" %i for i in leg_base]
    coxa_end_annotate = ["%0.4f" %i for i in coxa_end]
    femur_end_annotate = ["%0.4f" %i for i in femur_end]
    tibia_end_annotate = ["%0.4f" %i for i in tibia_end]

    print("Leg_base:", leg_base_annotate)
    print("coxa_end:", coxa_end_annotate)
    print("femur_end:", femur_end_annotate)
    print("tibia_end:", tibia_end_annotate)
    print("theta: %0.4f" %theta)

    coords_x = (leg_base[0], coxa_end[0], femur_end[0], tibia_end[0])
    coords_y = (leg_base[1], coxa_end[1], femur_end[1], tibia_end[1])
    coords_z = (leg_base[2], coxa_end[2], femur_end[2], tibia_end[2])

    
    leg_annotations = (leg_base_annotate, coxa_end_annotate, femur_end_annotate, tibia_end_annotate)

    # TODO print end coords onto visualization;
    # Plot both views
    fig, axs = plt.subplots(1,2)
    fig.suptitle("Leg IK simulation")
    axs[0].set_title("Top View")
    axs[0].set_xlabel("y")
    axs[0].set_xlabel("x")
    axs[0].plot(coords_y, coords_x, 'x-', linewidth=5)
    # axs[0].plot( (leg_base[1], coxa_end[1]), (leg_base[0], coxa_end[0]), 'x-', linewidth=5, label="Coxa")
    # axs[0].plot( (coxa_end[1], femur_end[1]), (coxa_end[0], femur_end[0]), 'x-', linewidth=5, label="Femur")
    # axs[0].plot( (femur_end[1], tibia_end[1]), (femur_end[0], tibia_end[0]), 'x-', linewidth=5, label="Tibia")
    for i, txt in enumerate(leg_annotations):
        axs[0].annotate(txt, (coords_y[i], coords_x[i]))
    axs[0].set_ylim(-45, 45)
    axs[0].set_xlim(0, 90)
    axs[0].grid()

    axs[1].set_title("Side view")
    axs[1].set_xlabel("y")  
    axs[1].set_xlabel("x")
    axs[1].plot(coords_y, coords_z, 'x-', linewidth=5)
    # axs[1].plot( (leg_base[1], coxa_end[1]), (leg_base[2], coxa_end[2]), 'x-', linewidth=5, label="Coxa")
    # axs[1].plot( (coxa_end[1], femur_end[1]), (coxa_end[2], femur_end[2]), 'x-', linewidth=5, label="Femur")
    # axs[1].plot( (femur_end[1], tibia_end[1]), (femur_end[2], tibia_end[2]), 'x-', linewidth=5, label="Tibia")
    for i, txt in enumerate(leg_annotations):
        axs[1].annotate(txt, (coords_y[i], coords_z[i]))
    axs[1].set_ylim(-30, 90)
    axs[1].set_xlim(0, 120)
    axs[1].grid()

    plt.tight_layout()
    plt.show()

    return [leg_base, coxa_end, femur_end, tibia_end, theta]

In [77]:
# Swing: Leg origin does not move; leg tip moves
def swing(tip_coords):
    coxa_angle_ik = np.arctan2(tip_coords[0], tip_coords[1]) # 'gamma' in notes

    coxa_len_ik = coxa_len * np.cos( coxa_angle_ik ) # Takes side view of this thing.
    femur_len_ik = femur_len * np.cos( coxa_angle_ik )
    tibia_len_ik = tibia_len * np.cos( coxa_angle_ik )

    y_offset_ik = tip_coords[1] - coxa_len_ik
    z_offset_ik = z_offset - tip_coords[2]
    l_len = np.sqrt( z_offset_ik**2 + y_offset_ik**2 )

    print("z_offset_ik: %0.4f | y_offset_ik: %0.4f | l_len: %0.4f" %(z_offset_ik, y_offset_ik, l_len) )

    femur_angle_ik1 = np.arccos( z_offset_ik / l_len )
    # print(z_offset_ik / l_len)
    femur_angle_ik2 = np.arccos( (femur_len_ik**2 + l_len**2 - tibia_len_ik**2) / (2*femur_len_ik*l_len) )
    # print((femur_len**2 + l_len**2 - tibia_len**2) / (2*femur_len*l_len))
    # print(to_degs(femur_angle_ik1), to_degs(femur_angle_ik2))
    femur_angle_ik = femur_angle_ik1 + femur_angle_ik2

    tibia_angle_ik = np.arccos( (femur_len_ik**2 - l_len**2 + tibia_len_ik**2) / (2*femur_len_ik*tibia_len_ik) )
    # print((femur_len**2 - l_len**2 + tibia_len**2) / (2*femur_len*tibia_len))

    angles_ik = [to_degs(coxa_angle_ik)+90, to_degs(femur_angle_ik), to_degs(tibia_angle_ik)]

    return angles_ik

In [31]:
# Stance: Leg tip does not move, leg origin moves.
# Modify the swing IK to frame leg tip in terms of origin
# Origin is at (0,0,68.825)
def stance(origin_coords):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-31-3d9b4b4411a6>, line 5)

In [175]:
# Body IK
# Return a np array of coordinates. The arguments of these coordinates are relative to a neutral position.

def body_roll(origin_coords, roll):
    new_coords = np.copy( origin_coords )
    new_coords[:, 1] = origin_coords[:, 1] * np.cos( to_rads(roll) ) # Y
    new_coords[:, 2] += origin_coords[:, 1] * np.sin( to_rads(roll) ) # Z

    print("origin_coords:\n", origin_coords)
    print("new_coords:\n", new_coords)
    
    return new_coords

def body_pitch(origin_coords, pitch):
    new_coords = np.copy( origin_coords )
    new_coords[:, 0] = origin_coords[:, 0] * np.cos( to_rads(pitch) ) # X
    new_coords[:, 2] -= origin_coords[:, 0] * np.sin( to_rads(pitch) ) # Z (L2 and R2 are unchanged)
    new_coords[1, 2] = origin_coords[1, 2]
    new_coords[4, 2] = origin_coords[4, 2]

    print("origin_coords:\n", origin_coords)
    print("new_coords:\n", new_coords)

    return new_coords

def body_rotate(origin_coords, theta):
    new_coords = np.copy( origin_coords )
    distances = np.empty(0)
    angles = np.empty(0)
    for x in range(origin_coords.shape[0]):
        distances = np.append(distances, pythagoras( origin_coords[x, 0:2] ) )
        angles = np.append(angles, to_degs(np.arctan2( origin_coords[x, 1], origin_coords[x, 0]))+theta )
    print("distances:\n", distances)
    print("angles:\n", angles)

    new_coords[:, 0] = distances * np.cos( to_rads( angles ) )
    new_coords[:, 1] = distances * np.sin( to_rads( angles ) )

    print("origin_coords:\n", origin_coords)
    print("new_coords:\n", new_coords)

    return new_coords

def body_translate_x(origin_coords, x):
    new_coords = np.copy( origin_coords )
    new_coords[:, 0] += x
    print("origin_coords:\n", origin_coords)
    print("new_coords:\n", new_coords)

    return new_coords

def body_translate_y(origin_coords, y):
    new_coords = np.copy( origin_coords )
    new_coords[:, 1] += y
    print("origin_coords:\n", origin_coords)
    print("new_coords:\n", new_coords)
    
    return new_coords

In [194]:
# Visualise body ik
X0_LEN = 45.768
Y0_LEN = 26.424
Y1_LEN = 52.848
Z_OFFSET = 68.825
R1_ORIGIN = np.array( (X0_LEN, Y0_LEN, Z_OFFSET) )
R2_ORIGIN = np.array( (0.0, Y1_LEN, Z_OFFSET) )
R3_ORIGIN = np.array( (-X0_LEN, Y0_LEN, Z_OFFSET) )
L1_ORIGIN = np.array( (X0_LEN, -Y0_LEN, Z_OFFSET) )
L2_ORIGIN = np.array( (0.0, -Y1_LEN, Z_OFFSET) )
L3_ORIGIN = np.array( (-X0_LEN, -Y0_LEN, Z_OFFSET) )
leg_origins = np.array( (R1_ORIGIN, R2_ORIGIN, R3_ORIGIN, L3_ORIGIN, L2_ORIGIN, L1_ORIGIN) )

def plot_body(body_coords, origin_coords=leg_origins):

    coords_x = np.append( body_coords[:, 0], body_coords[0,0] )
    coords_y = np.append( body_coords[:, 1], body_coords[0,1] )
    coords_z = np.append( body_coords[:, 2], body_coords[0,2] )

    coords_x_origin = np.append( origin_coords[:, 0], origin_coords[0,0] )
    coords_y_origin = np.append( origin_coords[:, 1], origin_coords[0,1] )
    coords_z_origin = np.append( origin_coords[:, 2], origin_coords[0,2] )

    leg_labels = ['R1', 'R2', 'R3', 'L3', 'L2', 'L1', '' ]
    leg_labels_o = ['o_R1', 'o_R2', 'o_R3', 'o_L3', 'o_L2', 'o_L1', '' ]

    fig, axs = plt.subplots(1,3, figsize=(15,5))
    fig.suptitle("Body IK simulation")

    axs[0].set_title("Front view")
    axs[0].set_xlabel("y")  
    axs[0].set_ylabel("z")
    axs[0].plot(coords_y_origin, coords_z_origin, 'x-', linewidth=5)
    axs[0].plot(coords_y, coords_z, 'x-', linewidth=5)
    for i, txt in enumerate(leg_labels):
        axs[0].annotate(txt, (coords_y[i], coords_z[i]))
    for i, txt in enumerate(leg_labels_o):
        axs[0].annotate(txt, (coords_y_origin[i], coords_z_origin[i]+5))
    axs[0].set_xlim(-60, 60)
    axs[0].set_ylim(0, 120)
    axs[0].grid()

    axs[1].set_title("Side View")
    axs[1].set_xlabel("x")
    axs[1].set_ylabel("z")
    axs[1].plot(coords_x_origin, coords_z_origin, 'x-', linewidth=5)
    axs[1].plot(coords_x, coords_z, 'x-', linewidth=5)
    for i, txt in enumerate(leg_labels):
        axs[1].annotate(txt, (coords_x[i], coords_z[i]))
    for i, txt in enumerate(leg_labels_o):
        axs[1].annotate(txt, (coords_x_origin[i], coords_z_origin[i]+5))
    axs[1].set_xlim(-60, 60)
    axs[1].set_ylim(0, 120)
    axs[1].grid()

    axs[2].set_title("Top View")
    axs[2].set_ylabel("x")
    axs[2].set_xlabel("y")
    axs[2].plot(coords_y_origin, coords_x_origin, 'x-', linewidth=5)
    axs[2].plot(coords_y, coords_x, 'x-', linewidth=5)
    for i, txt in enumerate(leg_labels):
        axs[2].annotate(txt, (coords_y[i], coords_x[i]))
    for i, txt in enumerate(leg_labels_o):
        axs[2].annotate(txt, (coords_y_origin[i], coords_x_origin[i]-2))
    axs[2].set_xlim(-60, 60)
    axs[2].set_ylim(-60, 60)
    axs[2].grid()

    plt.tight_layout()
    plt.show()

origin_coords:
 [[ 45.768  26.424  68.825]
 [  0.     52.848  68.825]
 [-45.768  26.424  68.825]
 [-45.768 -26.424  68.825]
 [  0.    -52.848  68.825]
 [ 45.768 -26.424  68.825]]
new_coords:
 [[ 45.768  36.424  68.825]
 [  0.     62.848  68.825]
 [-45.768  36.424  68.825]
 [-45.768 -16.424  68.825]
 [  0.    -42.848  68.825]
 [ 45.768 -16.424  68.825]]


In [130]:
leg_end_coords = [0.0, 68.75, 0.0] # check back

_ = plot_legs(swing(leg_end_coords))

z_offset_ik: 68.8250 | y_offset_ik: 40.0000 | l_len: 79.6045

Desired angles: ['90.0000', '90.0000', '90.0000']
Coxa length: 28.75
Femur length: 40.0
Tibia length: 68.825
Leg_base: ['0.0000', '0.0000', '68.8250']
coxa_end: ['0.0000', '28.7500', '68.8250']
femur_end: ['0.0000', '68.7500', '68.8250']
tibia_end: ['0.0000', '68.7500', '-0.0000']
theta: 0.0000


In [35]:
# Body kinematics simulator: